In [65]:
import time
import cv2
import mss
import numpy as np
import os
import sys
import tensorflow as tf
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from PIL import Image
import time

In [66]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [67]:
PATH_TO_FROZEN_GRAPH = path_of_the_model 
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'Images_training/labelmap.pbtxt'
NUM_CLASSES = no_of_classes

In [68]:
# ## Load a (frozen) Tensorflow model into memory.
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [69]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [70]:
test = []
files = os.listdir("TEST/")
for name in files:
    test.append(name)

In [71]:
start = time.process_time()
with detection_graph.as_default():
    with tf.Session(graph = detection_graph) as sess:
        for file in test:
            path = os.path.join("TEST/",file)
            title = file
            image = Image.open(path)# the array based representation of the image will be used later in order to prepare the# result image with boxes and labels on it.
            image_np = np.array(image)# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis = 0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')# Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')# Each score represent how level of confidence for each of the objects.#Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')# Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict = {
                      image_tensor: image_np_expanded
                    })
            
            vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8, min_score_thresh=.2)
            plt.title(title)
            plt.imshow(image_np)
            plt.show()
            ay = file.split(".")
            name_res = ay[0]+"_result.jpg"
            save_path = os.path.join("TEST/",name_res)
            img_res = Image.fromarray(image_np, 'RGB')
            img_res.save(save_path)

time_taken = time.process_time() - start
   

In [72]:
total_image = len(test)
inference_time = total_image / time_taken
print("Inference Time :",inference_time)

Inference Time : 0.09621166566446182
